In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import polars as pl
import seaborn as sns
from make_clinical_dataset.epic.preprocess.lab import clean_lab_data
from make_clinical_dataset.epic.util import load_lab_map, plot_count_over_time
from make_clinical_dataset.shared.constants import INFO_DIR, ROOT_DIR

pl.Config.set_fmt_str_lengths(100)
pl.Config.set_tbl_rows(200)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [ ]:
date = '2025-03-29'

# Raw

In [ ]:
# get the lab name mapping
lab_map = load_lab_map(data_dir=INFO_DIR)

# get the mrn mapping
mrn_map = pd.read_csv(f'{INFO_DIR}/mrn_map.csv')
mrn_map = mrn_map.set_index('PATIENT_RESEARCH_ID')['MRN'].to_dict()

In [ ]:
df = pl.scan_parquet(f'{ROOT_DIR}/data/processed/lab/lab_{date}/*.parquet')
df = clean_lab_data(df, mrn_map, lab_map)

## units

In [ ]:
df.group_by('obs_name', 'obs_unit').len().sort('obs_name').collect()

In [ ]:
# distribution bar plot for each unit
import matplotlib.pyplot as plt
import numpy as np
summary = (
    df
    .group_by("obs_name", "obs_unit")
    .agg([pl.col("obs_val_num").quantile(q).alias(f"q{q:0.2f}") for q in np.arange(0, 1, 0.01)])
)
summary = summary.sort('obs_name').collect().to_pandas()

In [ ]:
n = summary['obs_name'].nunique()
nrows, ncols = n // 2, 2
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*5, nrows*5))
axes = axes.flatten()
for i, (name, group) in enumerate(summary.groupby('obs_name')):
    for _, row in group.iterrows():
        unit = row['obs_unit'] 
        if unit is None: unit = 'null'
        vals = row.iloc[2:].to_list()
        ticks = row.iloc[2:].index.to_list()
        axes[i].bar(ticks, vals, label=unit, alpha=0.5)
        axes[i].set_xticks(['q0.01', 'q0.25', 'q0.50', 'q0.75', 'q0.99'])
        axes[i].set_title(name, fontdict={'fontsize': 10, 'fontweight': 'medium'})
        axes[i].legend()

In [ ]:
summary

## string entries

In [ ]:
tmp = df.filter(pl.col('obs_val_str').is_not_null()).collect()
tmp.group_by('proc_name', 'obs_val_str', 'obs_name', 'orig_obs_name').len().sort('len', descending=True)

In [ ]:
num_col, str_col = "obs_val_num", "obs_val_str"
df.select(
    (pl.col(num_col).is_not_null() & pl.col(str_col).is_null()).sum().alias('Only numerical entries'),
    (pl.col(num_col).is_null() & pl.col(str_col).is_null()).sum().alias('Neither'),
    (pl.col(num_col).is_not_null() & pl.col(str_col).is_not_null()).sum().alias('Both'),
    (pl.col(num_col).is_null() & pl.col(str_col).is_not_null()).sum().alias('Only string entries')
).collect()

## datetime

In [ ]:
main_date_col, secondary_date_col = "effective_datetime", "occurrence_datetime_from_order"
df.select(
    (pl.col(main_date_col).is_not_null() & pl.col(secondary_date_col).is_null()).sum().alias('Only effective_datetime'),
    (pl.col(main_date_col).is_null() & pl.col(secondary_date_col).is_null()).sum().alias('Neither'),
    (pl.col(main_date_col).is_not_null() & pl.col(secondary_date_col).is_not_null()).sum().alias('Both'),
    (pl.col(main_date_col).is_null() & pl.col(secondary_date_col).is_not_null()).sum().alias('Only occurence_datetime_from_order')
).collect()

In [ ]:
both = df.filter(pl.col(main_date_col).is_not_null() & pl.col(secondary_date_col).is_not_null())

In [ ]:
# how often effective_datetime is earlier than occurence_datetime_from_order
mask = pl.col(main_date_col) < pl.col(secondary_date_col)
both.select(mask.value_counts().alias('earlier')).collect()

In [ ]:
# distribution of the differences
diff = pl.col(main_date_col) - pl.col(secondary_date_col)
diff = both.select(diff.alias('diff')).collect().to_pandas()
diff['diff'].dt.days.value_counts().head(100).sort_index()

# Processed

In [ ]:
df = pd.read_parquet(f'{ROOT_DIR}/data/final/data_{date}/interim/lab.parquet')
df['obs_year'] = pd.to_datetime(df['obs_date']).dt.year

In [ ]:
N = df['mrn'].nunique()
min_date, max_date = df['obs_date'].min(), df['obs_date'].max()
print(f'{N} patients from {min_date} to {max_date}')

In [ ]:
# measurement counts over time
cols = df.columns.drop(['mrn', 'obs_year', 'obs_date'])
counts = df.groupby('obs_year').apply(lambda g: g[cols].notnull().sum(), include_groups=False)
counts = counts.reset_index().melt('obs_year', var_name='lab_test', value_name='count')
plot_count_over_time(counts, x='obs_year', y='count', catcol='lab_test', col_wrap=3)

In [ ]:
# overall missingness
df.isnull().mean().sort_values()

In [ ]:
# patients over time
df.groupby('obs_year')['mrn'].nunique().plot(kind='bar')

In [ ]:
# tests per patients
df.groupby('obs_year').apply(
    lambda g: g.groupby('mrn').apply(len, include_groups=False).mean(),
    include_groups=False
).plot(kind='bar')

In [ ]:
%%time
from ydata_profiling import ProfileReport
profile = ProfileReport(df, minimal=True)
profile.to_file("reports/lab.html")
profile.to_notebook_iframe()